In [1]:
import pandas as pd
import numpy as np 
from plotly import graph_objects as go
import matplotlib.pyplot as plt
from pathlib import Path

import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf 

import json
import os
from random import randint
import random
import shutil
from numpy import array
from numpy import argmax
from numpy import array_equal

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping


2025-01-24 08:56:11.382439: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 08:56:12.262087: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737708972.577605    1512 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737708972.676005    1512 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 08:56:13.379219: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Preprocessing

In [2]:
# Configuration de l'ImageDataGenerator avec preprocessing_function

img_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,  # Normalisation des pixels
    #preprocessing_function=preprocess_grayscale_to_rgb,  # Fonction de conversion des canaux
    validation_split=0.2  # Fraction pour la validation
    #rotation_range=90,
    #brightness_range=(0.5,1), 
    #shear_range=0.2, 
    #zoom_range=0.2,
    #channel_shift_range=0.2,
    #horizontal_flip=True,
    #vertical_flip=True
)

In [3]:
# Chargement des ensembles
BATCH_SIZE = 64
#img_size = (256, 256)   # taille originelle du dataset
img_size = (128, 128)  # pour faire une réduction de taille : de 256,256 à 128,128
seed = 42

In [4]:
# Chargement des données  de train avec flow_from_directory
img_generator_flow_train = img_generator.flow_from_directory(
    directory="../../../../Main_dataset_Sample_Binaire",  # chemin vers les données
    target_size=img_size,    # taille des images
    batch_size=BATCH_SIZE,
    shuffle=True,
    color_mode="grayscale",    # pour charger les images en niveaux de gris
    #color_mode="rgb",    # pour charger les images en niveaux de gris
    class_mode = "binary",#sparse
    #class_mode="categorical",  # Pour une classification multi-classes
    subset="training",
    seed = seed
    )

# Chargement des données  de validation avec flow_from_directory
img_generator_flow_valid = img_generator.flow_from_directory(
    directory="../../../../Main_dataset_Sample_Binaire",
    target_size=img_size,
    batch_size=BATCH_SIZE,
    shuffle=False,
    color_mode="rgb", #"grayscale"
    subset="validation",
    class_mode = "binary",#sparse
    #class_mode="categorical",
    seed= seed
    )

Found 3684 images belonging to 2 classes.
Found 920 images belonging to 2 classes.


In [5]:
## Définir les poids de classe 
from sklearn.utils.class_weight import compute_class_weight
class_indices = img_generator_flow_train.class_indices
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(list(class_indices.values())),
    y=img_generator_flow_train.classes
)
class_weights = dict(enumerate(class_weights))

# code donné par Malika :
# classes = np.unique(y_train) 
# class_weights = compute_class_weight('balanced', classes=classes, y=y_train)

# # Convertir en dict pour Keras
# class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}
# print("Class Weights:", class_weight_dict)

Model Inception v3

Fit without fine-tuning

In [13]:
base_model = tf.keras.applications.InceptionV3(input_shape=(128,128,3),
                                               include_top=False,
                                               weights = "imagenet"
                                               )
base_model.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [15]:
model_binary_simple = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(128, 128, 1)),  # Entrée en niveaux de gris
    tf.keras.layers.Conv2D(3, (1, 1), padding='same', use_bias=False, kernel_initializer='ones', trainable=False),  # Conversion 1 -> 3 canaux pour le base_model
    base_model,
    #tf.keras.layers.GlobalMaxPooling2D(),
    #tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_binary_simple.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_96 (Conv2D)              │ (None, 128, 128, 3)    │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 32768)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │        32,769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,620,484 (90.10 MB)

 Trainable params: 32,769 (128.00 KB)

 Non-trainable params: 23,587,715 (89.98 MB)

In [16]:
# Let's create a learning rate schedule to decrease the learning rate as we train the model.
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)

# Créer un optimiseur avec le planning du taux d'apprentissage
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [17]:
model_binary_simple.compile(
              #optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              optimizer=optimizer,
              loss = tf.keras.losses.BinaryCrossentropy(),
              #loss=weighted_binary_crossentropy(class_weights),#d_cat),
              metrics = [tf.keras.metrics.BinaryAccuracy()])

In [18]:
# Définir un callback d'early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Surveiller la perte de validation
    patience=3,          # Nombre d'époques sans amélioration avant d'arrêter
    restore_best_weights=True  # Rétablir les poids du meilleur modèle
)

In [11]:
# Activer le mode eager execution
tf.config.run_functions_eagerly(True)

In [19]:
import time
starttime = time.time

# Entraîner le modèle avec early stopping
history = model_binary_simple.fit(
    img_generator_flow_train,
    validation_data=img_generator_flow_valid,
    epochs=50,  # Plus d'époques pour laisser l'early stopping décider
    steps_per_epoch=len(img_generator_flow_train),
    validation_steps=len(img_generator_flow_valid),
    class_weight=class_weights,
    callbacks=[early_stopping]  #  callback 
)

stoptime = time.time
print(stoptime - starttime)

Epoch 1/50


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


58/58 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - binary_accuracy: 0.5114 - loss: 0.7322

ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "conv2d_96" is incompatible with the layer: expected axis -1 of input shape to have value 1, but received input with shape (64, 128, 128, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(64, 128, 128, 3), dtype=float32)
  • training=False
  • mask=None

In [ ]:
#model_binary_simple.save("./Binary_Inception_woFineTun.keras")

Fit with fine-tuning

In [7]:
len(base_model.layers)

311

In [ ]:
base_model = tf.keras.applications.InceptionV3(input_shape=(128,128,3),
                                               include_top=False,
                                               weights = "imagenet"
                                               )

base_model.trainable = True

fine_tune_at = len(base_model.layers) - 30
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False
for layer in base_model.layers[fine_tune_at:]:
  layer.trainable = True

In [ ]:
model_binary_finetuned = tf.keras.Sequential([
    base_model,
    #tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_binary_finetuned.summary()

In [ ]:
model_binary_finetuned.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.BinaryAccuracy()])

In [ ]:
import time
starttime = time.time
model_binary_finetuned.fit(img_generator_flow_train, validation_data=img_generator_flow_valid, epochs=10, # steps_per_epoch=20,
                class_weight=d_cat)
# par défaut, steps_per_epoch= taille du dataset / taille des batchs, en arrondissant à l'inférieur
stoptime = time.time
print(stoptime - starttime)

In [ ]:
#model_binary_finetuned.save("./Binary_Inception_withFineTun.keras")

Interpretation with GradCam

In [ ]:
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

imgs, targets = next(iter(img_generator_flow_valid))

In [ ]:
classifier_layer_names = [layer.name for layer in model_binary_finetuned.layers][1:]
print(classifier_layer_names)

last_conv_layer_name = base_model.layers[-1].name
print(last_conv_layer_name)

In [ ]:
from tensorflow import keras
# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
# The Grad-CAM algorithm
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(
    img_array, base_model, model, last_conv_layer_name, classifier_layer_names):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = base_model.get_layer(last_conv_layer_name)
    last_conv_layer_model = tf.keras.Model(base_model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [ ]:
preds = model_binary_finetuned.predict(imgs)
print(preds)

In [ ]:
# à modifier
pred_labels = tf.argmax(preds, axis=-1)
print(pred_labels)

In [ ]:
  # Generate class activation heatmap
heatmaps = []
for img in imgs:
    heatmap = make_gradcam_heatmap(
      tf.expand_dims(img,axis=0), base_model, model_binary_finetuned, last_conv_layer_name, classifier_layer_names
    )
    heatmaps.append(heatmap)

# Display heatmap
plt.matshow(heatmaps[0])
plt.show()

In [ ]:
num_rows = 4
num_cols = 8
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 15))

#Parcours des images / labels / heatmaps
for idx, (img, pred_label, true_label, heatmap) in enumerate(
    zip(imgs, pred_labels, labels, heatmaps)
):
    # Normalisation de la heatmap
    heatmap = np.uint8(255 * heatmap)

#Application de la cmap "jet"
    jet = plt.colormaps.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Redimensionnement de la heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superposition
    superimposed_img = jet_heatmap * 0.003 + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

    # Récup et affichage de pred et true
    pred_file_path = np.argmax(img_generator_flow_valid.labels == pred_label)
    pred_label_name = Path(img_generator_flow_valid.filepaths[pred_file_path]).parent.name
    true_file_path = np.argmax(img_generator_flow_valid.labels == tf.argmax(true_label))
    true_label_name = Path(img_generator_flow_valid.filepaths[true_file_path]).parent.name

    # Positionnement de l'image dans la grille
    ax = axes[idx // num_cols, idx % num_cols]

#Affichage
    ax.imshow(superimposed_img)
    ax.axis("off")
    ax.set_title(f"Pred: {pred_label_name}\nTrue: {true_label_name}", fontsize=8)

plt.tight_layout()
plt.show()